# Crawler for centaur.delegateselect.com

- [Customer search  page](https://centaur.delegateselect.com/MT/Run/event/main.html;jsessionid=9C11F56845D0A7C1C25E50152958BC99?Menu=attendees&Action=Start)

In [4]:
#Configs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [164]:
# imports
import requests
from bs4 import BeautifulSoup
import csv

## Explore data 

In [10]:
# session id is refreshed every x minutes and is required to make requests
session_id = '81AC054E08FFCD41388BE5BA3722E462'

# index start for the result payload(always 15 results)
company_index = 1

# when the sessionid is expired, a request will return:
# b'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'
url =  f'https://centaur.delegateselect.com/MT/Run/event/attendees.html;jsessionid={ session_id }?AJAX=1&PSVal=1'

In [11]:
# site content with the first 15 companies payload
site_content = requests.get(url).content

In [12]:
soup = BeautifulSoup(site_content)

In [201]:
# explore a data sample
x = soup.find_all('a')[1]
x.contents
x.attrs
x.has_attr('onclick')
x.attrs.get('onclick')
x.decode_contents()

['XL Travel (Pty) Ltd / Flight Specials']

{'href': 'javascript://',
 'onclick': "showProfile('157918','profile.html;jsessionid=FE8303267C70E8C0DF4DB06F272CE0C0')"}

True

"showProfile('157918','profile.html;jsessionid=FE8303267C70E8C0DF4DB06F272CE0C0')"

'XL Travel (Pty) Ltd / Flight Specials'

In [223]:
# let's move and find the job title for the company name
y = x.parent.parent.findNextSibling(class_='row')
y.find_next().decode_contents()

'\n\t\t\t\tGeneral Manager\n\t\t\t'

In [148]:
# let's start eating the soup
result = []

links_search = soup.find_all('a')

In [149]:
# let's get only the a tags that have a showProfile method call onclick event. that means it
for item in links_search:
    if item.has_attr('onclick') and item.attrs.get('onclick')[:11].startswith('showProfile'):
        result.append(item)

In [150]:
result

[<a href="javascript://" onclick="showProfile('157964','profile.html;jsessionid=81AC054E08FFCD41388BE5BA3722E462')">247 vip ltd</a>,
 <a href="javascript://" onclick="showProfile('158069','profile.html;jsessionid=81AC054E08FFCD41388BE5BA3722E462')">2RNJOY</a>,
 <a href="javascript://" onclick="showProfile('158119','profile.html;jsessionid=81AC054E08FFCD41388BE5BA3722E462')">360 Private Travel</a>,
 <a href="javascript://" onclick="showProfile('157779','profile.html;jsessionid=81AC054E08FFCD41388BE5BA3722E462')">3Sixty Digital</a>,
 <a href="javascript://" onclick="showProfile('158505','profile.html;jsessionid=81AC054E08FFCD41388BE5BA3722E462')">Aamal Travel</a>,
 <a href="javascript://" onclick="showProfile('157806','profile.html;jsessionid=81AC054E08FFCD41388BE5BA3722E462')">Abercrombie &amp; Kent</a>,
 <a href="javascript://" onclick="showProfile('157821','profile.html;jsessionid=81AC054E08FFCD41388BE5BA3722E462')">Abercrombie &amp; Kent Ltd</a>,
 <a href="javascript://" onclick="sho

In [151]:
list_of_companies = []

for item in result:
    list_of_companies.append({
        'name': item.decode_contents()
    })


In [152]:
list_of_companies
f'len: { len(list_of_companies) }'

[{'name': '247 vip ltd'},
 {'name': '2RNJOY'},
 {'name': '360 Private Travel'},
 {'name': '3Sixty Digital'},
 {'name': 'Aamal Travel'},
 {'name': 'Abercrombie &amp; Kent'},
 {'name': 'Abercrombie &amp; Kent Ltd'},
 {'name': 'ABRAJ ALBET TRAVEL&amp; TOURISM SERVICES'},
 {'name': 'AeroCrest Travel Network'},
 {'name': 'AFY Travel'},
 {'name': 'agency'},
 {'name': 'Agoda.com'},
 {'name': 'Air France-KLM'},
 {'name': 'Air Partner TMC'},
 {'name': 'AirBing'}]

'len: 15'

## Start the scrapping process 

In [231]:
# session id is refreshed every x minutes and is required to make requests
session_id = 'FE8303267C70E8C0DF4DB06F272CE0C0'

soup_filter_results = []

# each request gives us 15 results and we have 46 pages so let's iterate over them starting at 1 and finishing at 676, the last group of 15 results 
for company_index in range(1, 676+15, 15):
    
    url =  f'https://centaur.delegateselect.com/MT/Run/event/attendees.html;jsessionid={ session_id }?AJAX=1&PSVal={ company_index }'
    
    # site content with the first 15 companies payload
    site_content = requests.get(url).content
    
    # start soup
    soup = BeautifulSoup(site_content)
    
    # cook the soup
    soup_of_a_tags = soup.find_all('a')
    
    # let's get only the a tags that have a showProfile method call onclick event. that means it
    for item in soup_of_a_tags:
        if item.has_attr('onclick') and item.attrs.get('onclick')[:11].startswith('showProfile'):
            soup_filter_results.append(item)


# create dict of companies
list_of_companies = []

for item in soup_filter_results:
    
    # find the job title for the company entry
    job_title = item.parent.parent.findNextSibling(class_='row').find_next().decode_contents()
    # remove newline and space chars 
    job_title = ''.join(job_title.split())
    
    # add to results
    list_of_companies.append({
        'name': item.decode_contents().replace('&amp;', ' &'),
        'job_title': job_title.replace('&amp;', '&')
    })

In [232]:
'First 10 sample data:' 
list_of_companies[:10]
f'Processed {len(list_of_companies) } companies'

'First 10 sample data:'

[{'name': '247 vip ltd', 'job_title': 'director'},
 {'name': '2RNJOY', 'job_title': 'CEO'},
 {'name': '360 Private Travel', 'job_title': 'CEO'},
 {'name': '3Sixty Digital', 'job_title': 'Director'},
 {'name': 'Aamal Travel', 'job_title': 'GeneralManager'},
 {'name': 'Abercrombie  & Kent', 'job_title': 'SolutionsArchitect'},
 {'name': 'Abercrombie  & Kent Ltd', 'job_title': 'SupportEngineer'},
 {'name': 'ABRAJ ALBET TRAVEL & TOURISM SERVICES',
  'job_title': 'GENERALMANAGER'},
 {'name': 'AeroCrest Travel Network', 'job_title': 'Founder&CEO'},
 {'name': 'AFY Travel', 'job_title': 'MemberoftheBoard'}]

'Processed 679 companies'

In [233]:
%%capture
# will capture all output and won't display it on the cell output 

# store soup in a csv file
with open('./centaur_attendees.csv', 'w', newline='') as csv_file:
    field_names = ['name', 'job_title']
    writer = csv.DictWriter(csv_file, fieldnames=field_names)
    
    writer.writeheader()
    
    for company in list_of_companies:
        writer.writerow(company)